**SQL ASSESSMENT 2**

You are given a history of your bank account transactions for the year 2020. Each transaction was either aa credit card payment or an incoming transfer. 
There is a fee for holding a credit card which you have to pay every month. 
Tthe cost you are charged each month is 5. 
However, you are not charged for a given month if you made at least three credit card payments for a total costt
of at least 100 within that month but this fee is not included in the transaction history.

Need to pay $5 if:

(1) Less than 3 credit card transactions per month

(2) The transactions are less than $100

Ans: -612

In [6]:
import pandas as pd
import sqlite3
from pandas.io import sql # reads sql statement and selects data from database

Account_transaction = pd.read_csv("ACCOUNT TRANSACTION.csv")# import csv files 
Account_transaction
Account_transaction.sort_values(by='DATE')

,AMOUNT,DATE
2,-50,2020-02-03
5,-44,2020-02-07
1,35,2020-02-20
6,-5,2020-02-25
3,-1,2020-02-26
0,1,2020-06-29
7,1,2020-06-29
8,1,2020-06-29
4,-200,2020-08-01
9,-100,2020-12-29


In [7]:
Account_transaction.head()

,AMOUNT,DATE
0,1,2020-06-29
1,35,2020-02-20
2,-50,2020-02-03
3,-1,2020-02-26
4,-200,2020-08-01


In [8]:
db_sql_2 = sqlite3.connect('sql_assessment_task_2.db.sqlite') #creating empty sql database
Account_transaction.to_sql('BALANCES',con = db_sql_2, if_exists = 'replace', index = False) #we are converting the dataframe into a sql database, con means connecting to the dataframe and replacing it, index means dropping 1,2,3 etc. first column

In [9]:
query = """
        SELECT SUM(AMOUNT) AS TOTAL_AMOUNT, strftime('%m',DATE) AS new_month
        FROM BALANCES
        GROUP BY new_month
        """
NEW_ACCOUNT_TRANSACTION = sql.read_sql(query,con =db_sql_2)
NEW_ACCOUNT_TRANSACTION

,TOTAL_AMOUNT,new_month
0,-65,02
1,3,06
2,-200,08
3,-300,12


In [53]:
query = """
       
       SELECT (FINAL_AMOUNT+FINAL_CREDIT_CARD_CHARGE+ANNUAL_CREDIT_CARD_CHARGE) AS BALANCE
       
       FROM
       
       (SELECT SUM(TOTAL_AMOUNT) AS FINAL_AMOUNT, SUM(CREDIT_CARD_CHARGE) AS FINAL_CREDIT_CARD_CHARGE, (-5*12) AS ANNUAL_CREDIT_CARD_CHARGE 
       FROM
       
       (SELECT NUMBER_OF_CREDIT_CARD_TRANSACTIONS, 
       TOTAL_AMOUNT,
       CREDIT_CARD_TRANSACTION_VALUES,
       new_month,

       CASE WHEN NUMBER_OF_CREDIT_CARD_TRANSACTIONS < 3 AND CREDIT_CARD_TRANSACTION_VALUES <= -100 OR CREDIT_CARD_TRANSACTION_VALUES IS NULL THEN '5'
       ELSE NULL END AS CREDIT_CARD_CHARGE

       FROM
       (SELECT SUM(AMOUNT) AS TOTAL_AMOUNT, SUM(case when AMOUNT<0 then 1 else 0 end) as NUMBER_OF_CREDIT_CARD_TRANSACTIONS,strftime('%m',DATE) AS new_month, sum(case when amount<0 then amount end) AS CREDIT_CARD_TRANSACTION_VALUES, SUM(AMOUNT) as AMOUNT_PER_MONTH     
       FROM BALANCES      
       GROUP BY new_month)p )z)r
       
              
        """

NEW_ACCOUNT_TRANSACTION = sql.read_sql(query,con =db_sql_2)
NEW_ACCOUNT_TRANSACTION


,BALANCE
0,-612


In [54]:
query = """
       
       SELECT SUM(TOTAL_AMOUNT + CREDIT_CARD_CHARGE + ANNUAL_CREDIT_CARD_CHARGE) AS BALANCE, CREDIT_CARD_CHARGE, ANNUAL_CREDIT_CARD_CHARGE   
       FROM
       
       (SELECT NUMBER_OF_CREDIT_CARD_TRANSACTIONS, 
       TOTAL_AMOUNT,
       CREDIT_CARD_TRANSACTION_VALUES,
       new_month,
       (-5 *12) AS ANNUAL_CREDIT_CARD_CHARGE,

       CASE WHEN NUMBER_OF_CREDIT_CARD_TRANSACTIONS < 3 AND CREDIT_CARD_TRANSACTION_VALUES <= -100 OR CREDIT_CARD_TRANSACTION_VALUES IS NULL THEN '5'
       ELSE NULL END AS CREDIT_CARD_CHARGE

       FROM
       (SELECT SUM(AMOUNT) AS TOTAL_AMOUNT, SUM(case when AMOUNT<0 then 1 else 0 end) as NUMBER_OF_CREDIT_CARD_TRANSACTIONS,strftime('%m',DATE) AS new_month, sum(case when amount<0 then amount end) AS CREDIT_CARD_TRANSACTION_VALUES, SUM(AMOUNT) as AMOUNT_PER_MONTH     
       FROM BALANCES      
       GROUP BY new_month)p)z
       
               """

NEW_ACCOUNT_TRANSACTION = sql.read_sql(query,con =db_sql_2)
NEW_ACCOUNT_TRANSACTION

,BALANCE,CREDIT_CARD_CHARGE,ANNUAL_CREDIT_CARD_CHARGE
0,-307,None,-60


In [36]:
query = """
       
       SELECT NUMBER_OF_CREDIT_CARD_TRANSACTIONS, 
       TOTAL_AMOUNT,
       CREDIT_CARD_TRANSACTION_VALUES,
       new_month,

       CASE WHEN NUMBER_OF_CREDIT_CARD_TRANSACTIONS < 3 AND CREDIT_CARD_TRANSACTION_VALUES <= -100  OR CREDIT_CARD_TRANSACTION_VALUES IS NULL THEN '-5'
       ELSE NULL END AS CREDIT_CARD_CHARGE

       FROM
       (SELECT SUM(AMOUNT) AS TOTAL_AMOUNT, SUM(case when AMOUNT<0 then 1 else 0 end) as NUMBER_OF_CREDIT_CARD_TRANSACTIONS,strftime('%m',DATE) AS new_month, sum(case when amount<0 then amount end) AS CREDIT_CARD_TRANSACTION_VALUES, SUM(AMOUNT) as AMOUNT_PER_MONTH     
       FROM BALANCES      
       GROUP BY new_month)p 
       
              
        """

NEW_ACCOUNT_TRANSACTION = sql.read_sql(query,con =db_sql_2)
NEW_ACCOUNT_TRANSACTION


,NUMBER_OF_CREDIT_CARD_TRANSACTIONS,TOTAL_AMOUNT,CREDIT_CARD_TRANSACTION_VALUES,new_month,CREDIT_CARD_CHARGE
0,4,-65,-100.0,02,None
1,0,3,NaN,06,-5
2,1,-200,-200.0,08,-5
3,3,-300,-300.0,12,None
